In [40]:
import re
import json
import pickle
import pandas as pd
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.corpus.reader.wordnet import WordNetError

In [47]:
with open('testAlign.pkl', 'rb') as inp:
    testAlign = pickle.load(inp)

In [5]:
with open('en.test_predictions.txt') as enTest:
    senseList = enTest.readlines()

In [21]:
def keyParsing(total_key):
    key_regex = r"d(.*).s(.*).t(.*) (.*)"
    doc, sen, term, senseID = re.match(key_regex, total_key).groups()
    
    return sen, senseID

In [41]:
sense_key_regex = r"(.*)\%(.*):(.*):(.*):(.*):(.*)"
synset_types = {1:'n', 2:'v', 3:'a', 4:'r', 5:'s'}

def synset_from_sense_key(sense_key):
    
    lemma, ss_type, lex_num, lex_id, head_word, head_id = re.match(sense_key_regex, sense_key).groups()
    ss_idx = '.'.join([lemma, synset_types[int(ss_type)], lex_id])
    
    try:
        synset = wn.synset(ss_idx)
    except WordNetError:
        synset = ss_idx
       
    return lemma, synset

In [79]:
temp="Synset('alike.r.02')"

In [81]:
synset_regex = r"Synset\('(.*)'\)"
synsetFi = re.match(synset_regex, temp).group(1)

In [82]:
synsetFi

'alike.r.02'

In [57]:
testAlign[0][2][0][1]

'Number'

In [86]:
multi_WN = []
lemmatizer = WordNetLemmatizer()


for sense in senseList:
    
    sen, senseID = keyParsing(sense)
    idx = int(sen)
    
    try:
        lemma, synsetWN = synset_from_sense_key(senseID)
    except AttributeError:
        pass 

    for i in range(len(testAlign[idx])):
        ele = testAlign[idx][i]
        temp = ele[1].lower()
        lema_ele = lemmatizer.lemmatize(temp)
        
        if lema_ele == lemma:
            if ele[0][1] not in ['Adverb', 'Noun', 'Verb', 'Adjective']:
                pass
            
            else:
                synTemp = str(synsetWN)
                try:
                    synset_regex = r"Synset\('(.*)'\)"
                    synsetFi = re.match(synset_regex, synTemp).group(1)
                except AttributeError:
                    synsetFi = synTemp
                
                sense_dic = {}
                sense_dic['src'] = temp
                sense_dic['trg'] = ele[0][0]
                sense_dic['lemma'] = lemma
                sense_dic['sense_key'] = senseID
                synsetTemp = str(synsetWN)
                sense_dic['wordnet_synset'] = synsetFi
                sense_dic['location'] = int(sen)
                multi_WN.append(sense_dic)

In [89]:
len(multi_WN)

10123

In [88]:
with open("Multi_Wordnet.json", "w", encoding='utf-8') as multiWN:
    json.dump(multi_WN, multiWN, indent=4)